In [1]:
%matplotlib inline
from __future__ import division

import pandas as pd
import utils

In [2]:
visitors, events, devices, url_categories = utils.load_data()

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5,6,7,8,11,12,13,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [4]:
event_categories = utils.categorize_events(events, url_categories)

match rate: 99.0%


In [6]:
categorized_visits = pd.merge(visitors, event_categories, on='dg_id')

In [8]:
categorized_visits['authorized'] = categorized_visits.identity_type.apply(lambda x: 1 if x=='AUT' else 0)

In [35]:
pharma = categorized_visits[categorized_visits.site_category=='Pharma']

In [36]:
pharma_activity = pharma.groupby('url_y').authorized.agg(['sum', 'count']).reset_index()
pharma_activity.columns = ['url', 'authorized_visits', 'total_visits']

In [37]:
pharma_activity = pd.merge(url_categories, pharma_activity, on='url')

In [38]:
pharma_activity['pct_authorized'] = pharma_activity.apply(lambda x: x.authorized_visits/x.total_visits, axis=1)

In [39]:
pharma_activity

,url,site_category,site_sub_category,disease_category,disease,pharma_firm,stripped_url,clean_url,authorized_visits,total_visits,pct_authorized
0,http://www.adempas-us.com,Pharma,Branded Consumer,"Cardiology, Pulmonology\rMultiple","Thromboembolic Pulmonary Hypertension, Pulmona...",Bayer,adempas-us,adempas-us.com,48,63,0.761905
1,http://www.adempas-us.com/hcp/,Pharma,Branded HCP,"Cardiology, Pulmonology\rMultiple","Thromboembolic Pulmonary Hypertension, Pulmona...",Bayer,adempas-us/hcp,adempas-us.com/hcp,50,62,0.806452
2,http://www.herceptin.com/hcp/,Pharma,Branded HCP,Oncology,"Metastatic Breast Cancer, Metastatic Gastric C...",Genentech,herceptin/hcp,herceptin.com/hcp,79,133,0.593985
3,http://www.journeyforcontrol.com/,Pharma,Unbranded HCP,Endocrinology,Diabetes,Merck,journeyforcontrol,journeyforcontrol.com,903,1274,0.708791
4,http://www.kadcyla.com/hcp/,Pharma,Branded HCP,Oncology,metastatic breast cancer,Genentech,kadcyla/hcp,kadcyla.com/hcp,82,112,0.732143
5,http://www.lenvima.com/hcp/,Pharma,Branded HCP,Oncology,"Thyroid Cancer, advanced renal cell carcinoma ...",Eisai,lenvima/hcp,lenvima.com/hcp,297,334,0.889222
6,http://www.perjeta.com/hcp/,Pharma,Branded HCP,Oncology,metastatic breast cancer,Genentech,perjeta/hcp,perjeta.com/hcp,469,646,0.726006
7,http://www.pomalyst.com/hcp/,Pharma,Branded HCP,Oncology,multiple myeloma,Celgene,pomalyst/hcp,pomalyst.com/hcp,168,199,0.844221
8,http://www.rituxan.com/hem/hcp,Pharma,Branded HCP,"Oncology, Rheumatology\rMultiple","non-Hodgkin's lymphoma, �chronic lymphocytic l...",Genentech,rituxan/hcp,rituxan.com/hem/hcp,124,170,0.729412
9,http://www.tarceva.com/hcp/,Pharma,Branded HCP,Oncology,"non-small cell lung cancer (NSCLC), advanced-s...",Genentech,tarceva/hcp,tarceva.com/hcp,59,88,0.670455


In [123]:
def authorized_visits(pharma_site, pharma_activity):
    f = pharma_activity[pharma_activity.url==pharma_site]
    print 'Pharma Site :: ' + pharma_site + ' :: ' + str(round(f.pct_authorized*100,2)) + '% authorized'
    disease_category = f.disease_category.item()
    fd = pharma_activity[pharma_activity.disease_category==disease_category]
    if len(fd) >= 3:
        authorized = sum(fd.authorized_visits)
        total = sum(fd.total_visits)
        print 'Disease Category :: ' + disease_category + ' :: ' + str(round(authorized/total*100, 2)) + '% authorized'
        diseases = [x.lower().strip() for x in f.disease.item().split(',') if x.strip() != '']
        for d in diseases:
            authorized = 0
            total = 0
            count = 0
            for row in fd.iterrows():
                r = row[1]
                if d in r.disease.lower():
                    authorized += r.authorized_visits
                    total += r.total_visits
                    count += 1
            if count >= 3:
                print 'Disease :: ' + d + ' :: ' + str(round(authorized/total*100, 2)) + '% authorized'
                

In [128]:
for url in pharma_activity.url.sample():
    authorized_visits(url, pharma_activity)

Pharma Site :: www.AstellasClinicalStudyResults.com :: 27.27% authorized
Disease Category :: Multiple :: 73.98% authorized
